In [2]:
import numpy as np
import sympy as sym
import control as ctrl
from matplotlib import pyplot as plt

In [3]:
g=9.81  # m/s^2
L=0.125 # distance from pendulum's centre of mass to pivot in m
a=0.016 # radius of pulley in m
m=0.32  # mass of pendulum in kg
M=0.7  # mass of carriage in kg
I=8e-5  # moment of inertia on motor shaft in kg m^2
km = 0.08  # torque motor constant in Nm/A
ka = -0.50 # amplifier constant in A/V

gamma =  M/m + I/(m*np.power(a,2))

In [4]:

omega_1 = sym.symbols('omega_1', real=True)
omega_0 = sym.symbols('omega_0', real=True)

Ac = sym.Matrix(4,4, [0,1,0,0,
                        0,0,omega_1**2 - omega_0**2, 0,
                        0,0,0,1,
                        0,0,-omega_0**2, 0])

B = sym.Matrix(4,1, [0,1,0,1])
Sc=np.diag([-1/12.5, -1/2.23, L/3.18, L/0.64])
opamp_c = np.diag([-20,-30, 20, -10]) # for crane controller
Cc=-(ka*km/(m*a*gamma))*np.linalg.solve(Sc,opamp_c)

s = sym.symbols('s')
t = sym.symbols('t')

k1,k2,k3,k4 = sym.symbols('k1 k2 k3 k4', real=True)

K =  sym.Matrix(1,4, [k1,k2,k3,k4])

cG_ = sym.eye(4)*s - Ac
Gc_sys = Cc * cG_.inv() * B

cG_ = sym.eye(4)*s - Ac + B*K
crane_poly = sym.poly(sym.det(cG_), s)
Gc_closed = Cc * cG_.inv() * B

#xc = sympy.inverse_laplace_transform(Gc / s, s, t)

# factorise and convert to latex
print("Closed loop characteristic polynomial for crane model:")
print(sym.latex(crane_poly))

print("Closed loop transfer function for crane model:")
print(sym.latex(sym.simplify(Gc_closed)))

Closed loop characteristic polynomial for crane model:
\operatorname{Poly}{\left( s^{4} + \left(k_{2} + k_{4}\right) s^{3} + \left(k_{1} + k_{3} + \omega_{0}^{2}\right) s^{2} + k_{2} \omega_{1}^{2} s + k_{1} \omega_{1}^{2}, s, domain=\mathbb{Z}\left[k_{1}, k_{2}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right] \right)}
Closed loop transfer function for crane model:
\left[\begin{matrix}\frac{617.283950617284 \left(\omega_{1}^{2} + s^{2}\right)}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\\frac{165.185185185185 s \left(\omega_{1}^{2} + s^{2}\right)}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\\frac{1256.2962962963 s^{2}}{k_{1} \omega_{1}^{2} + k_{1} s^{2} + k_{2} \omega_{1}^{2} s + k_{2} s^{3} + k_{3} s^{2} + k_{4} s^{3} + \omega_{0}^{2} s^{2} + s^{4}}\\- \frac{126.41975308642 s^{3}}{k_{1} \omega_{1}^{2} + k_{1} 

for n = 4:
$a_i > 0 , \quad a_1a_2a_3 > a_0a_3^2 + a_4a_1^2$

In [5]:
# get routh hourwitz conditions

#print(dir(crane_poly))

crane_coeffs = crane_poly.all_coeffs()

for i in range(4):
    print(sym.latex(crane_coeffs[i]), "> 0")

crane_left_inequality = crane_coeffs[1] * crane_coeffs[2] * crane_coeffs[3]
crane_right_inequality = crane_coeffs[0] * crane_coeffs[3] ** 2 + crane_coeffs[4] * crane_coeffs[1] ** 2
crane_inequality_expr = crane_left_inequality.expand() - crane_right_inequality.expand()

crane_inequality_quadratic = sym.poly(crane_inequality_expr / omega_1**2, k2)

print(sym.latex(crane_inequality_quadratic))

crane_k2 = sym.solve(crane_inequality_quadratic, k2)[0] # only 0 satisfies previous RH criteria

print("LaTeX Solutions for k2 inequality")
print(sym.latex(crane_k2))

# now compute poles from k2



1 > 0
k_{2} + k_{4} > 0
k_{1} + k_{3} + \omega_{0}^{2} > 0
k_{2} \omega_{1}^{2} > 0
\operatorname{Poly}{\left( \left(k_{3} + \omega_{0}^{2} - \omega_{1}^{2}\right) k_{2}^{2} + \left(- k_{1} k_{4} + k_{3} k_{4} + k_{4} \omega_{0}^{2}\right) k_{2} -  k_{1} k_{4}^{2}, k_{2}, domain=\mathbb{Z}\left(k_{1}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right) \right)}
LaTeX Solutions for k2 inequality
\frac{k_{4} \left(k_{1} - k_{3} - \omega_{0}^{2} - \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}\right)}{2 \left(k_{3} + \omega_{0}^{2} - \omega_{1}^{2}\right)}


In [6]:

crane_poly_factor = s**2 + k2 / (k2 + k4) * omega_1**2

crane_poly_substituted = crane_poly.subs({k2 : crane_k2})

factor, remainder = sym.div(crane_poly_substituted, crane_poly_factor)

print(sym.latex(remainder))

- \frac{k_{1} k_{2} k_{4} \omega_{1}^{2} s}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{1} k_{2} \omega_{1}^{2}}{k_{2} + k_{4}} + \frac{k_{1} k_{4} \omega_{1}^{2} s}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} + k_{1} \omega_{1}^{2} + \frac{k_{2}^{2} \omega_{1}^{4}}{\left(k_{2} + k_{4}\right)^{2}} + \frac{k_{2} k_{3} k_{4} \omega_{1}^{2} s}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{3} \omega_{1}^{2}}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{0}^{2} \omega_{1}^{2} s}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{4} \omega_{1}^{2} s}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{1}^{2} s \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac

$ - \frac{k_{1} k_{2} k_{4} \omega_{1}^{2} s}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{1} k_{2} \omega_{1}^{2}}{k_{2} + k_{4}} + \frac{k_{1} k_{4} \omega_{1}^{2} s}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} + k_{1} \omega_{1}^{2} + \frac{k_{2}^{2} \omega_{1}^{4}}{\left(k_{2} + k_{4}\right)^{2}} + \frac{k_{2} k_{3} k_{4} \omega_{1}^{2} s}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{3} \omega_{1}^{2}}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{0}^{2} \omega_{1}^{2} s}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{4} \omega_{1}^{2} s}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{1}^{2} s \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} \omega_{0}^{2} \omega_{1}^{2}}{k_{2} + k_{4}} - \frac{k_{3} k_{4} \omega_{1}^{2} s}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} - \frac{k_{4} \omega_{0}^{2} \omega_{1}^{2} s}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} - \frac{k_{4} \omega_{1}^{2} s \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}}$

In [7]:
crane_k1 = sym.solve(crane_inequality_quadratic, k2)[0] # only 0 satisfies previous RH criteria

crane_poly_substituted = crane_poly.subs({k1 : crane_k1})

factor, remainder = sym.div(crane_poly_substituted, crane_poly_factor)

print(sym.latex(remainder))

- \frac{k_{1} k_{2} k_{4} \omega_{1}^{2}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} + \frac{k_{1} k_{4} \omega_{1}^{2}}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} + \frac{k_{2}^{2} \omega_{1}^{4}}{\left(k_{2} + k_{4}\right)^{2}} - \frac{k_{2}^{2} \omega_{1}^{2} s}{k_{2} + k_{4}} + \frac{k_{2} k_{3} k_{4} \omega_{1}^{2}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{3} \omega_{1}^{2}}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{0}^{2} \omega_{1}^{2}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{4} \omega_{1}^{2} s}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{1}^{2} \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} \omega_{0}^{2} \omega_{1}^

$ - \frac{k_{1} k_{2} k_{4} \omega_{1}^{2}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} + \frac{k_{1} k_{4} \omega_{1}^{2}}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} + \frac{k_{2}^{2} \omega_{1}^{4}}{\left(k_{2} + k_{4}\right)^{2}} - \frac{k_{2}^{2} \omega_{1}^{2} s}{k_{2} + k_{4}} + \frac{k_{2} k_{3} k_{4} \omega_{1}^{2}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{3} \omega_{1}^{2}}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{0}^{2} \omega_{1}^{2}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} k_{4} \omega_{1}^{2} s}{k_{2} + k_{4}} + \frac{k_{2} k_{4} \omega_{1}^{2} \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}}{\left(k_{2} + k_{4}\right) \left(2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}\right)} - \frac{k_{2} \omega_{0}^{2} \omega_{1}^{2}}{k_{2} + k_{4}} + k_{2} \omega_{1}^{2} s - \frac{k_{3} k_{4} \omega_{1}^{2}}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} - \frac{k_{4} \omega_{0}^{2} \omega_{1}^{2}}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} - \frac{k_{4} \omega_{1}^{2} \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}}{2 k_{3} + 2 \omega_{0}^{2} - 2 \omega_{1}^{2}} $

In [8]:
omega_hat = sym.symbols('\hat{\omega}', real=True)
crane_poly_substituted = crane_poly.subs({s : 1j * omega_hat})

imag_part = sym.im(crane_poly_substituted.as_expr())
real_part = sym.re(crane_poly_substituted.as_expr())

print(sym.latex(imag_part))
print(sym.latex(real_part))

freq_response = sym.solve(imag_part, omega_hat)

real_part.subs({omega_hat : freq_response[1]}).factor()

- 1.0 \hat{\omega}^{3} \left(k_{2} + k_{4}\right) + 1.0 \hat{\omega} k_{2} \omega_{1}^{2}
1.0 \hat{\omega}^{4} - 1.0 \hat{\omega}^{2} \left(k_{1} + k_{3} + \omega_{0}^{2}\right) + k_{1} \omega_{1}^{2}


1.0*omega_1**2*(1.0*k1*k2*k4 + 1.0*k1*k4**2 - 1.0*k2**2*k3 - 1.0*k2**2*omega_0**2 + 1.0*k2**2*omega_1**2 - 1.0*k2*k3*k4 - 1.0*k2*k4*omega_0**2)/(k2 + k4)**2

you are a bastard

$\operatorname{Poly}{\left( 1.0 \hat{\omega}^{4} + \left(k_{2} + k_{4}\right) - 1.0 i \hat{\omega}^{3} + \left(k_{1} + k_{3} + \omega_{0}^{2}\right) - 1.0 \hat{\omega}^{2} + k_{2} \omega_{1}^{2} 1.0 i \hat{\omega} + k_{1} \omega_{1}^{2}, 1.0 i \hat{\omega}, domain=\mathbb{Z}\left[k_{1}, k_{2}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right] \right)}$

In [9]:
Ap = sym.Matrix(4,4, [0,1,0,0,
                        0,0,omega_0**2 - omega_1**2, 0,
                        0,0,0,1,
                        0,0,omega_0**2, 0])
opamp_p = np.diag([10,20,30,-20]) # for inverted pendulum controller
Sp=np.diag([-1/12.5, -1/2.23, L/3.18, -L/0.64])
Cp=-(ka*km/(m*a*gamma))*np.linalg.solve(Sp,opamp_p)

pG_ = sym.eye(4)*s - Ap
Gp_sys = Cp * pG_.inv() * B

pG_ = sym.eye(4)*s - Ap + B*K
pendulum_poly = sym.poly(sym.det(pG_), s)

Gp_closed = Cc * pG_.inv() * B

In [10]:

pendulum_coeffs = pendulum_poly.all_coeffs()

for i in range(4):
    print(sym.latex(pendulum_coeffs[i]), "> 0")

pendulum_left_inequality = pendulum_coeffs[1] * pendulum_coeffs[2] * pendulum_coeffs[3]
pendulum_right_inequality = pendulum_coeffs[0] * pendulum_coeffs[3] ** 2 + pendulum_coeffs[4] * pendulum_coeffs[1] ** 2
pendulum_inequality_expr = pendulum_left_inequality.expand() - pendulum_right_inequality.expand()

pendulum_inequality_quadratic = sym.poly(pendulum_inequality_expr / omega_1**2, k2)

print(sym.latex(pendulum_inequality_quadratic))

pendulum_k2 = sym.solve(pendulum_inequality_quadratic, k2)[0] # only 0 satisfies previous RH criteria

print("LaTeX Solutions for k2 inequality")
print(sym.latex(crane_k2))

# now compute poles from k2

1 > 0
k_{2} + k_{4} > 0
k_{1} + k_{3} - \omega_{0}^{2} > 0
- k_{2} \omega_{1}^{2} > 0
\operatorname{Poly}{\left( \left(- k_{3} + \omega_{0}^{2} - \omega_{1}^{2}\right) k_{2}^{2} + \left(k_{1} k_{4} - k_{3} k_{4} + k_{4} \omega_{0}^{2}\right) k_{2} + k_{1} k_{4}^{2}, k_{2}, domain=\mathbb{Z}\left(k_{1}, k_{3}, k_{4}, \omega_{0}, \omega_{1}\right) \right)}
LaTeX Solutions for k2 inequality
\frac{k_{4} \left(k_{1} - k_{3} - \omega_{0}^{2} - \sqrt{k_{1}^{2} + 2 k_{1} k_{3} + 2 k_{1} \omega_{0}^{2} - 4 k_{1} \omega_{1}^{2} + k_{3}^{2} + 2 k_{3} \omega_{0}^{2} + \omega_{0}^{4}}\right)}{2 \left(k_{3} + \omega_{0}^{2} - \omega_{1}^{2}\right)}


In [11]:

pendulum_poly_substituted = pendulum_poly.subs({s : 1j * omega_hat})

imag_part = sym.im(pendulum_poly_substituted.as_expr())
real_part = sym.re(pendulum_poly_substituted.as_expr())

freq_response = sym.solve(imag_part, omega_hat)

print(sym.latex(imag_part))
print(sym.latex(real_part))

print(sym.latex(freq_response[2]))

real_part.subs({omega_hat : freq_response[2]}).factor()

- 1.0 \hat{\omega}^{3} \left(k_{2} + k_{4}\right) - 1.0 \hat{\omega} k_{2} \omega_{1}^{2}
1.0 \hat{\omega}^{4} - 1.0 \hat{\omega}^{2} \left(k_{1} + k_{3} - \omega_{0}^{2}\right) - k_{1} \omega_{1}^{2}
\omega_{1} \sqrt{- \frac{k_{2}}{k_{2} + k_{4}}}


-1.0*omega_1**2*(1.0*k1*k2*k4 + 1.0*k1*k4**2 - 1.0*k2**2*k3 + 1.0*k2**2*omega_0**2 - 1.0*k2**2*omega_1**2 - 1.0*k2*k3*k4 + 1.0*k2*k4*omega_0**2)/(k2 + k4)**2

In [34]:
# controllability PENDULUM

Ap_m = Ap

# if controllable
controllability_matrix = sym.Matrix.hstack(B, Ap_m*B, Ap_m**2*B, Ap_m**3*B)


# check rank is 4
print(controllability_matrix.rank())

print(sym.latex(controllability_matrix))

4
\left[\begin{matrix}0 & 1 & 0 & \omega_{0}^{2} - \omega_{1}^{2}\\1 & 0 & \omega_{0}^{2} - \omega_{1}^{2} & 0\\0 & 1 & 0 & \omega_{0}^{2}\\1 & 0 & \omega_{0}^{2} & 0\end{matrix}\right]


In [36]:
# controllability CRANE


# if controllable
controllability_matrix = sym.Matrix.hstack(B, Ac*B, Ac**2*B, Ac**3*B)

# check rank is 4
print(controllability_matrix.rank())
print(sym.latex(controllability_matrix))

4
\left[\begin{matrix}0 & 1 & 0 & - \omega_{0}^{2} + \omega_{1}^{2}\\1 & 0 & - \omega_{0}^{2} + \omega_{1}^{2} & 0\\0 & 1 & 0 & - \omega_{0}^{2}\\1 & 0 & - \omega_{0}^{2} & 0\end{matrix}\right]
